In [1]:
import tensorflow as tf
import gym
import numpy as np

### Variables
num_inputs = 4
num_hidden = 4
num_outputs = 1

learning_rate = 0.01

initializer = tf.contrib.layers.variance_scaling_initializer()


### Create networks
X = tf.placeholder(tf.float32, shape=[None, num_inputs])

hidden_layer = tf.layers.dense(X, num_hidden, activation=tf.nn.elu, kernel_initializer=initializer)
logits = tf.layers.dense(hidden_layer, num_outputs)
outputs = tf.nn.sigmoid(logits)  # probability of action 0 (left)

probabilties = tf.concat(axis=1, values=[outputs, 1 - outputs])
action = tf.multinomial( probabilties, num_samples=1)

y = 1. - tf.to_float(action)


### Lost func and Optimizer
cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=logits)
optimizer = tf.train.AdamOptimizer(learning_rate)



### Gradient
gradients_and_variables = optimizer.compute_gradients(cross_entropy)



gradients = []
gradient_placeholders = []
grads_and_vars_feed = []

for gradient, variable in gradients_and_variables:
    gradients.append(gradient)
    gradient_placeholder = tf.placeholder(tf.float32, shape=gradient.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable))


training_op = optimizer.apply_gradients(grads_and_vars_feed)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [2]:
### Reward functions ###

def helper_discount_rewards(rewards, discount_rate):
    '''
    Takes in rewards and applies discount rate
    '''
    discounted_rewards = np.zeros(len(rewards))
    cumulative_rewards = 0
    for step in reversed(range(len(rewards))):
        cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
        discounted_rewards[step] = cumulative_rewards
    return discounted_rewards

def discount_and_normalize_rewards(all_rewards, discount_rate):
    '''
    Takes in all rewards, applies helper_discount function and then normalizes
    using mean and std.
    '''
    all_discounted_rewards = []
    for rewards in all_rewards:
        all_discounted_rewards.append(helper_discount_rewards(rewards,discount_rate))

    flat_rewards = np.concatenate(all_discounted_rewards)
    reward_mean = flat_rewards.mean()
    reward_std = flat_rewards.std()
    return [(discounted_rewards - reward_mean)/reward_std for discounted_rewards in all_discounted_rewards]

In [3]:
### Training
env = gym.make("CartPole-v0")

num_game_rounds = 10
max_game_steps = 1000
num_iterations = 650
discount_rate = 0.95

with tf.Session() as sess:
    sess.run(init)


    for iteration in range(num_iterations):
        print("Currently on Iteration: {} \n".format(iteration) )

        all_rewards = []
        all_gradients = []

        # Play n amount of game rounds
        for game in range(num_game_rounds):

            current_rewards = []
            current_gradients = []

            observations = env.reset()

            # Only allow n amount of steps in game
            for step in range(max_game_steps):

                # Get Actions and Gradients
                action_val, gradients_val = sess.run([action, gradients], feed_dict={X: observations.reshape(1, num_inputs)})

                # Perform Action
                observations, reward, done, info = env.step(action_val[0][0])

                # Get Current Rewards and Gradients
                current_rewards.append(reward)
                current_gradients.append(gradients_val)

                if done:
                    # Game Ended
                    break

            # Append to list of all rewards
            all_rewards.append(current_rewards)
            all_gradients.append(current_gradients)

        all_rewards = discount_and_normalize_rewards(all_rewards,discount_rate)
        feed_dict = {}


        for var_index, gradient_placeholder in enumerate(gradient_placeholders):
            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]
                                      for game_index, rewards in enumerate(all_rewards)
                                          for step, reward in enumerate(rewards)], axis=0)
            feed_dict[gradient_placeholder] = mean_gradients

        sess.run(training_op, feed_dict=feed_dict)

    print('SAVING GRAPH AND SESSION')
    meta_graph_def = tf.train.export_meta_graph(filename='./models/rf-650-steps-model.meta')
    saver.save(sess, './models/rf-650-steps-model')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Currently on Iteration: 0 

Currently on Iteration: 1 

Currently on Iteration: 2 

Currently on Iteration: 3 

Currently on Iteration: 4 

Currently on Iteration: 5 

Currently on Iteration: 6 

Currently on Iteration: 7 

Currently on Iteration: 8 

Currently on Iteration: 9 

Currently on Iteration: 10 

Currently on Iteration: 11 

Currently on Iteration: 12 

Currently on Iteration: 13 

Currently on Iteration: 14 

Currently on Iteration: 15 

Currently on Iteration: 16 

Currently on Iteration: 17 

Currently on Iteration: 18 

Currently on Iteration: 19 

Currently on Iteration: 20 

Currently on Iteration: 21 

Currently on Iteration: 22 

Currently on Iteration: 23 

Currently on Iteration: 24 

Currently on Iteration: 25 

Currently on Iteration: 26 

Currently on Iteration: 27 

Currently on Iteration: 28 

Currently on Iteration: 29 

Currently on Iteration: 30 

Currently on

Currently on Iteration: 274 

Currently on Iteration: 275 

Currently on Iteration: 276 

Currently on Iteration: 277 

Currently on Iteration: 278 

Currently on Iteration: 279 

Currently on Iteration: 280 

Currently on Iteration: 281 

Currently on Iteration: 282 

Currently on Iteration: 283 

Currently on Iteration: 284 

Currently on Iteration: 285 

Currently on Iteration: 286 

Currently on Iteration: 287 

Currently on Iteration: 288 

Currently on Iteration: 289 

Currently on Iteration: 290 

Currently on Iteration: 291 

Currently on Iteration: 292 

Currently on Iteration: 293 

Currently on Iteration: 294 

Currently on Iteration: 295 

Currently on Iteration: 296 

Currently on Iteration: 297 

Currently on Iteration: 298 

Currently on Iteration: 299 

Currently on Iteration: 300 

Currently on Iteration: 301 

Currently on Iteration: 302 

Currently on Iteration: 303 

Currently on Iteration: 304 

Currently on Iteration: 305 

Currently on Iteration: 306 

Currently 

Currently on Iteration: 548 

Currently on Iteration: 549 

Currently on Iteration: 550 

Currently on Iteration: 551 

Currently on Iteration: 552 

Currently on Iteration: 553 

Currently on Iteration: 554 

Currently on Iteration: 555 

Currently on Iteration: 556 

Currently on Iteration: 557 

Currently on Iteration: 558 

Currently on Iteration: 559 

Currently on Iteration: 560 

Currently on Iteration: 561 

Currently on Iteration: 562 

Currently on Iteration: 563 

Currently on Iteration: 564 

Currently on Iteration: 565 

Currently on Iteration: 566 

Currently on Iteration: 567 

Currently on Iteration: 568 

Currently on Iteration: 569 

Currently on Iteration: 570 

Currently on Iteration: 571 

Currently on Iteration: 572 

Currently on Iteration: 573 

Currently on Iteration: 574 

Currently on Iteration: 575 

Currently on Iteration: 576 

Currently on Iteration: 577 

Currently on Iteration: 578 

Currently on Iteration: 579 

Currently on Iteration: 580 

Currently 

In [2]:
### Run model on env
env = gym.make('CartPole-v0')

observations = env.reset()
with tf.Session() as sess:
    # https://www.tensorflow.org/api_guides/python/meta_graph
    new_saver = tf.train.import_meta_graph('./models/rf-650-steps-model.meta')
    new_saver.restore(sess,'./models/rf-650-steps-model')

    for x in range(500):
        env.render()
        action_val, gradients_val = sess.run([action, gradients], feed_dict={X: observations.reshape(1, num_inputs)})
        observations, reward, done, info = env.step(action_val[0][0])

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from ./models/my-650-step-model
WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
